In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [4]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(39010, 2831) (12128, 2831)


In [5]:
features_list =[
#Bureau Intutive features
'Num_Enq_all_loans_last_9M_3M',
'Utilization_all',
'total_emi',
'Utilization_CREDIT CARD_opened_last_36M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'num_CREDIT CARD_enq_last_6M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'highest_sanction_amount_credit_cards',
'num_PL_LE_1_Lac_opened_in_last_12M',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M',
'Age_of_oldest_open_account',
'num_CONSUMER LOAN_enq_last_6M_36M',
    
#Banking Intutive features
    
'num_credit_gt_salary',    
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M1',    
'Min_Balance_Min_M1_M2_M3',
'ratio_max_credit_to_total_credit_M1',
'spend_vs_salary_ratio',
'bounced_transactions_count',
'ratio_total_debit_to_max_credit_M2_M3',
'min_balance_last_2_months',
'days_balance_lt_5000_M2',
'days_balance_lt_10000_M3',
'num_credits_M2',
'num_debits_M3',
'balance_3th_day_M1',
    
'Max_Credit_Avg_M1_M2_M3',
'Max_Credit_Diff_M1_M2_M3',
'amount_debits_last_2_months',
'sum_insurance_debit',
'avg_num_debits_last_1_months',
'upi_trans_debit_sum',
'avg_num_credits_last_3_months',
'invest_spending_pct',
'cash_withdrawal_sum',

    
# Reconsalited Features
'percentage_ever_delinquent', 
'percentage_utilization_gt_75',
'Num_of_installment_loan_opened_last_6M',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
    
'month_year',
'90_in_24mob'
              
]




In [6]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3023298/282645959.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3023298/282645959.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23406, 40), (15604, 40), (23406,), (15604,))

In [9]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 40) (4682, 40) (15604, 40)


In [10]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [11]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,

    'subsample': 0.6,
    'colsample_bytree': 0.6,

    'lambda': 7.5,
    'alpha':5.5,
    'gamma':4.5,

    'random_state': 24
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 700,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.59877	eval-auc:0.58663
[10]	train-auc:0.63032	eval-auc:0.62087
[20]	train-auc:0.64045	eval-auc:0.62754
[30]	train-auc:0.64283	eval-auc:0.63177
[40]	train-auc:0.64676	eval-auc:0.63595
[50]	train-auc:0.64744	eval-auc:0.63817
[60]	train-auc:0.64756	eval-auc:0.63767
[70]	train-auc:0.64929	eval-auc:0.63895
[80]	train-auc:0.65020	eval-auc:0.64027
[90]	train-auc:0.65046	eval-auc:0.64091
[100]	train-auc:0.65127	eval-auc:0.64240
[110]	train-auc:0.65211	eval-auc:0.64334
[120]	train-auc:0.65325	eval-auc:0.64472
[130]	train-auc:0.65403	eval-auc:0.64529
[140]	train-auc:0.65521	eval-auc:0.64602
[150]	train-auc:0.65650	eval-auc:0.64699
[160]	train-auc:0.65780	eval-auc:0.64770
[170]	train-auc:0.65871	eval-auc:0.64900
[180]	train-auc:0.65957	eval-auc:0.64920
[190]	train-auc:0.66018	eval-auc:0.64955
[200]	train-auc:0.66074	eval-auc:0.65049
[210]	train-auc:0.66165	eval-auc:0.65121
[220]	train-auc:0.66225	eval-auc:0.65140
[230]	train-auc:0.66311	eval-auc:0.65223
[240]	train-auc:0.66384	eva

In [12]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.69, Gini: 0.37, KS: 0.27
Eval AUC: 0.66, Gini: 0.32, KS: 0.25
Test AUC: 0.66, Gini: 0.33, KS: 0.24
OOT AUC: 0.67, Gini: 0.34, KS: 0.25


In [13]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.388681         0.337027        0.560657        0.269782   1873   
9          0.254274         0.241679        0.269780        0.218485   1872   
8          0.207265         0.202141        0.218483        0.187769   1872   
7          0.190603         0.175924        0.187769        0.165495   1873   
6          0.159188         0.156635        0.165482        0.148120   1872   
5          0.140491         0.140610        0.148118        0.133170   1872   
4          0.122264         0.127288        0.133169        0.120869   1873   
3          0.092415         0.114189        0.120857        0.107331   1872   
2          0.077991         0.102040        0.107329        0.094743   1872   
1          0.049119         0.083001        0.094733        0.057470   1873   

        label_good  label_bad  
Decile                         
10            1145        728  
9             1396        476  
8             1484        388  
7             1516        357  
6             1574        298  
5             1609        263  
4             1644        229  
3             1699        173  
2             1726        146  
1             1781         92

In [14]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.355541         0.334525        0.551816        0.269314   1561   
9          0.257692         0.240408        0.269131        0.217616   1560   
8          0.219231         0.201100        0.217584        0.187020   1560   
7          0.180013         0.175409        0.187003        0.164938   1561   
6          0.164744         0.155825        0.164936        0.147307   1560   
5          0.146154         0.140175        0.147307        0.133648   1560   
4          0.124920         0.127374        0.133612        0.120992   1561   
3          0.117308         0.114450        0.120987        0.107777   1560   
2          0.079487         0.102153        0.107776        0.094724   1560   
1          0.064061         0.082534        0.094694        0.054797   1561   

        label_good  label_bad  
Decile                         
10            1006        555  
9             1158        402  
8             1218        342  
7             1280        281  
6             1303        257  
5             1332        228  
4             1366        195  
3             1377        183  
2             1436        124  
1             1461        100

In [15]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.373454         0.336347        0.571003        0.269518   1213   
9          0.248145         0.241059        0.269497        0.217327   1213   
8          0.217642         0.201172        0.217250        0.186675   1213   
7          0.195545         0.175535        0.186658        0.164990   1212   
6          0.157461         0.156121        0.164983        0.147417   1213   
5          0.145095         0.140014        0.147416        0.133389   1213   
4          0.130363         0.127373        0.133376        0.121102   1212   
3          0.109646         0.114841        0.121101        0.108454   1213   
2          0.082440         0.103309        0.108452        0.096478   1213   
1          0.062655         0.084368        0.096476        0.054315   1213   

        label_good  label_bad  
Decile                         
10             760        453  
9              912        301  
8              949        264  
7              975        237  
6             1022        191  
5             1037        176  
4             1054        158  
3             1080        133  
2             1113        100  
1             1137         76

In [16]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                             Feature  Importance
7               highest_sanction_amount_credit_cards       198.0
10                        Age_of_oldest_open_account       157.0
0                       Num_Enq_all_loans_last_9M_3M       155.0
39                         Num_Enq_non_BANK_last_12M       130.0
34                               cash_withdrawal_sum       115.0
35                        percentage_ever_delinquent       103.0
1                                    Utilization_all        98.0
4           Utilization_PERSONAL LOAN_opened_last_9M        91.0
18                        bounced_transactions_count        80.0
11                 num_CONSUMER LOAN_enq_last_6M_36M        80.0
3            Utilization_CREDIT CARD_opened_last_36M        67.0
5                        num_CREDIT CARD_enq_last_6M        65.0
38                          No of enq in past 90days        65.0
21                           days_balance_lt_5000_M2        65.0
13                       

In [17]:
feature_importance_df

Feature  Importance
7               highest_sanction_amount_credit_cards       198.0
10                        Age_of_oldest_open_account       157.0
0                       Num_Enq_all_loans_last_9M_3M       155.0
39                         Num_Enq_non_BANK_last_12M       130.0
34                               cash_withdrawal_sum       115.0
35                        percentage_ever_delinquent       103.0
1                                    Utilization_all        98.0
4           Utilization_PERSONAL LOAN_opened_last_9M        91.0
18                        bounced_transactions_count        80.0
11                 num_CONSUMER LOAN_enq_last_6M_36M        80.0
3            Utilization_CREDIT CARD_opened_last_36M        67.0
5                        num_CREDIT CARD_enq_last_6M        65.0
38                          No of enq in past 90days        65.0
21                           days_balance_lt_5000_M2        65.0
13                                   upi_trans_count        55.0
6             Num_of_PERSONAL LOAN_opened_last6M_12M        50.0
17                             spend_vs_salary_ratio        43.0
20                         min_balance_last_2_months        36.0
8                 num_PL_LE_1_Lac_opened_in_last_12M        31.0
37            Num_of_installment_loan_opened_last_6M        31.0
2                                          total_emi        29.0
29                               sum_insurance_debit        27.0
9   Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M        24.0
22                          days_balance_lt_10000_M3        23.0
12                              num_credit_gt_salary        23.0
30                      avg_num_debits_last_1_months        22.0
15                          Min_Balance_Min_M1_M2_M3        20.0
36                      percentage_utilization_gt_75        13.0
31                               upi_trans_debit_sum        12.0
16               ratio_max_credit_to_total_credit_M1        12.0
14          ratio_balance_after_7_days_max_credit_M1         9.0
26                           Max_Credit_Avg_M1_M2_M3         9.0
25                                balance_3th_day_M1         8.0
24                                     num_debits_M3         8.0
27                          Max_Credit_Diff_M1_M2_M3         6.0
19             ratio_total_debit_to_max_credit_M2_M3         4.0
33                               invest_spending_pct         4.0
28                       amount_debits_last_2_months         3.0
23                                    num_credits_M2         2.0
32                     avg_num_credits_last_3_months         2.0

In [18]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BB_Train_Pred_9024.csv',index=False)
Eval.to_csv('LeoPayu_BB_Eval_Pred_9024.csv',index=False)
Test.to_csv('LeoPayu_BB_Test_Pred_9024.csv',index=False)
Oot.to_csv('LeoPayu_BB_Oot_Pred_9024.csv',index=False)


In [19]:
#Save the model
xgb_model.save_model('LeoPayu_BB_90_in_24mob.json')
print('Model Saved')

Model Saved
